# Magnetometer

The magnetic field sensor in the rocket is sensitive, but because the Earth's field is so weak it's easily overwhelmed by local effects (metal screws, magnetic fields from nearby wires, etc.). In order to get good orientation data we need to undo these local effects.

Before the flight we moved the rocket around in every direction to recored the magnetic field offset in each direction.

## Field Strength

First check, let's average the magnitude of the field and compare to what NOAA says it should have been for our location.

From [NOAA's magnetic field calculator](https://www.ngdc.noaa.gov/geomag/magfield.shtml)


 - Model Used: `WMM2015`
 - Latitude: `43.79613280° N`
 - Longitude: `120.65175340° W`
 - Elevation: `1390.0 m Mean Sea Level`

| Date | Declination (+E/-W) | Inclination (+D/-U) | Horizontal Intensity | North Comp (+N/-S) | East Comp (+E/-W) | Vertical Comp (+D/-U) | Total Field |
| ---- | ------------------- | ------------------- | -------------------- | ------------------ | -------------------- | --------------------- | ----------- | 
| 2015-07-17   | 14.7990° | 66.5386° | 20,754.1 nT | 20,065.7 nT | 5,301.2 nT | 47,819.4 nT | 52,129.0 nT | 
| Uncertainty  |    0.36° |    0.22° |      133 nT |      138 nT |      89 nT |      165 nT | 152 nT      |

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, sqrt, average
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665

columns = loadtxt("../cal-data/ADIS.csv", delimiter=',', unpack=True)
seqn = columns[0]
cal_time = divide(columns[1], 1e9)
gyro_x, gyro_y, gyro_z = columns[3:6]
acc_x, acc_y, acc_z = columns[6:9]
mag_x, mag_y, mag_z = columns[9:12]

cal_time = subtract(cal_time, cal_time[0])


# rest of flight data:
columns = loadtxt("../full-flight/ADIS.csv", delimiter=',', unpack=True)
seqn = columns[0]
flight_time = columns[1]
flightmag_x, flightmag_y, flightmag_z = columns[9:12]



def minsec(x, pos):
    m = x/60
    s = x - (m*60)
    return '%d:%02d' % (m,s)

# everything in microtesla
mag_x, mag_y, mag_z = multiply(mag_x, 1e6), multiply(mag_y, 1e6), multiply(mag_z, 1e6)
flightmag_x, flightmag_y, flightmag_z = multiply(flightmag_x, 1e6), multiply(flightmag_y, 1e6), multiply(flightmag_z, 1e6)

# Magnitude
mag = []
for i, t in enumerate(cal_time):
    mag.append(sqrt((mag_x[i]*mag_x[i]) + (mag_y[i]*mag_y[i]) + (mag_z[i]*mag_z[i])))


print """Our average total field strength measured %0.2f μT, compared to NOAA's 52.129 ± 0.152 μT.
""" % (average(mag))

We can also run a time series of the data and see how it changes. The total field strength shouldn't change, even as we move the rocket around.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Magnetometer Calibration Run")
plt.ylabel(r"Magnetic Field [$ \mu$T]")
plt.xlabel(r"Time [mm:ss]")

plt.plot(cal_time, mag, 'k-', alpha=0.3, label="Magnitude")
plt.plot([-100,5000], [52.129, 52.129], 'k-.', alpha=0.3, label="True Magnitude")
plt.plot(cal_time, mag_x, lw=0.5, label="X (Up)")
plt.plot(cal_time, mag_y, lw=0.5, label="Y")
plt.plot(cal_time, mag_z, lw=0.5, label="Z")

plt.xlim([0,1350])
plt.ylim([-100,100])
ax1.xaxis.set_major_formatter(FuncFormatter(minsec))
ax1.legend(loc=4)
plt.show()

It does change, because we have a big offset in some direction. The other way to look at this is a 3D plot of all the values. They _should_ land on a sphere, but instead it's an elongated ellipsoid.

In [ ]:
from matplotlib import gridspec
import matplotlib.patches as patches

fig = plt.figure(figsize=(16,16))
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1])

ax1 = plt.subplot(gs[0])
plt.title(r"XY")
plt.ylabel(r"Magnetic Field X [$ \mu$T]")
plt.xlabel(r"Magnetic Field Y [$ \mu$T]")
ax1.plot(mag_x, mag_y, lw=0.8, label="")
ax1.add_patch(patches.Circle((0, 0), 53, edgecolor="#cccccc", linewidth=1.0, linestyle='--', fill=False))
plt.xlim([-80,80])
plt.ylim([-80,80])

ax2 = plt.subplot(gs[1])
plt.title(r"YZ")
plt.ylabel(r"Magnetic Field Y [$ \mu$T]")
plt.xlabel(r"Magnetic Field Z [$ \mu$T]")
ax2.plot(mag_y, mag_z, lw=0.8, label="")
ax2.add_patch(patches.Circle((0, 0), 53, edgecolor="#cccccc", linewidth=1.0, linestyle='--', fill=False))
plt.xlim([-80,80])
plt.ylim([-80,80])

ax3 = plt.subplot(gs[2])
plt.title(r"ZX")
plt.ylabel(r"Magnetic Field Z [$ \mu$T]")
plt.xlabel(r"Magnetic Field X [$ \mu$T]")
ax3.plot(mag_z, mag_x, lw=0.8, label="")
ax3.add_patch(patches.Circle((0, 0), 53, edgecolor="#cccccc", linewidth=1.0, linestyle='--', fill=False))
plt.xlim([-80,80])
plt.ylim([-80,80])

plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from numpy import mgrid, pi, sin, cos
fig, ax1 = plt.subplots(figsize=(16,16))
ax1 = plt.axes(projection='3d')
plt.title(r"IMU Magnetometer vs 1 Earth Field Sphere")
plt.xlabel(r"Field Strength [$\mu$T]")
plt.ylabel(r"Field Strength [$\mu$T]")



u, v = mgrid[0:2*pi:40j, 0:pi:20j]
radius = 52.129
x=cos(u)*sin(v)*radius
y=sin(u)*sin(v)*radius
z=cos(v)*radius
ax1.plot_wireframe(x, y, z, color="b", alpha=0.4, lw=0.3)


ax1.plot(mag_x,mag_y,mag_z, '-')
ax1.plot(flightmag_x,flightmag_y,flightmag_z, '-')
ax1.plot([0],[0],[0], 'o')


ax1.set_xlim(-60, 60)
ax1.set_ylim(-60, 60)
ax1.set_zlim(-60, 60)

ax1.set_aspect('equal','box')
plt.show()

In [ ]:
"""http://teslabs.com/articles/magnetometer-calibration/"""
import numpy as np
from scipy import linalg

s = []
for i, t in enumerate(cal_time):
    s.append([mag_x[i], mag_y[i], mag_z[i]])

def ellipsoid_fit(s):
    ''' Estimate ellipsoid parameters from a set of points.

        Parameters
        ----------
        s : array_like
          The samples (M,N) where M=3 (x,y,z) and N=number of samples.

        Returns
        -------
        M, n, d : array_like, array_like, float
          The ellipsoid parameters M, n, d.

        References
        ----------
        .. [1] Qingde Li; Griffiths, J.G., "Least squares ellipsoid specific
           fitting," in Geometric Modeling and Processing, 2004.
           Proceedings, vol., no., pp.335-340, 2004
    '''

    # D (samples)
    D = np.array([s[0]**2., s[1]**2., s[2]**2.,
                  2.*s[1]*s[2], 2.*s[0]*s[2], 2.*s[0]*s[1],
                  2.*s[0], 2.*s[1], 2.*s[2], np.ones_like(s[0])])

    # S, S_11, S_12, S_21, S_22 (eq. 11)
    S = np.dot(D, D.T)
    S_11 = S[:6,:6]
    S_12 = S[:6,6:]
    S_21 = S[6:,:6]
    S_22 = S[6:,6:]

    # C (Eq. 8, k=4)
    C = np.array([[-1,  1,  1,  0,  0,  0],
                  [ 1, -1,  1,  0,  0,  0],
                  [ 1,  1, -1,  0,  0,  0],
                  [ 0,  0,  0, -4,  0,  0],
                  [ 0,  0,  0,  0, -4,  0],
                  [ 0,  0,  0,  0,  0, -4]])

    # v_1 (eq. 15, solution)
    E = np.dot(linalg.inv(C),
               S_11 - np.dot(S_12, np.dot(linalg.inv(S_22), S_21)))

    E_w, E_v = np.linalg.eig(E)

    v_1 = E_v[:, np.argmax(E_w)]
    if v_1[0] < 0: v_1 = -v_1

    # v_2 (eq. 13, solution)
    v_2 = np.dot(np.dot(-np.linalg.inv(S_22), S_21), v_1)

    # quadric-form parameters
    M = np.array([[v_1[0], v_1[3], v_1[4]],
                  [v_1[3], v_1[1], v_1[5]],
                  [v_1[4], v_1[5], v_1[2]]])
    n = np.array([[v_2[0]],
                  [v_2[1]],
                  [v_2[2]]])
    d = v_2[3]

    return M, n, d

M, n, d = ellipsoid_fit(np.array(s).T)

F = 52.129

M_1 = linalg.inv(M)
b = np.dot(M_1, n)
A_1 = np.real(F / np.sqrt(np.dot(n.T, np.dot(M_1, n)) - d) * linalg.sqrtm(M))

#print b
print "Correction Matrix:\n", A_1, ""

cmag_x = []
cmag_y = []
cmag_z = []

corrected_samples = []
for sample in s:
    #print sample
    sample = np.array(sample).reshape(3, 1)
    corrected = np.dot(A_1, sample - b)
    #print corrected
    #print [corrected[0,0], corrected[1,0], corrected[2,0]]
    cmag_x.append(float(corrected[0,0]))
    cmag_y.append(float(corrected[1,0]))
    cmag_z.append(float(corrected[2,0]))
    corrected_samples.append([corrected[0,0], corrected[1,0], corrected[2,0]])

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Magnetometer Calibration Run, Corrected")
plt.ylabel(r"Magnetic Field [$ \mu$T]")
plt.xlabel(r"Time [mm:ss]")

# Magnitude
mag = []
for i, t in enumerate(cal_time):
    mag.append(sqrt((cmag_x[i]*cmag_x[i]) + (cmag_y[i]*cmag_y[i]) + (cmag_z[i]*cmag_z[i])))

plt.plot(cal_time, mag, 'k-', alpha=0.3, label="Magnitude")
plt.plot([-100,5000], [52.129, 52.129], 'k-.', alpha=0.3, label="True Magnitude")
plt.plot(cal_time, cmag_x, lw=0.5, label="X (Up)")
plt.plot(cal_time, cmag_y, lw=0.5, label="Y")
plt.plot(cal_time, cmag_z, lw=0.5, label="Z")

plt.xlim([0,1350])
plt.ylim([-140,140])
ax1.xaxis.set_major_formatter(FuncFormatter(minsec))
ax1.legend(loc=4)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,16))
ax1 = plt.axes(projection='3d')
plt.title(r"IMU Magnetometer Corrected vs 1 Earth Field Sphere")
plt.xlabel(r"Field Strength [$\mu$T]")
plt.ylabel(r"Field Strength [$\mu$T]")

u, v = mgrid[0:2*pi:40j, 0:pi:20j]
radius = 52.129
x=cos(u)*sin(v)*radius
y=sin(u)*sin(v)*radius
z=cos(v)*radius
ax1.plot_wireframe(x, y, z, color="b", alpha=0.4, lw=0.3)

ax1.plot(cmag_x, cmag_y, cmag_z, '-')
ax1.plot(mag_x, mag_y, mag_z, '-', alpha=0.4)
#ax1.plot(flightmag_x,flightmag_y,flightmag_z, '-')
ax1.plot([0],[0],[0], 'o')

ax1.set_xlim(-60, 60)
ax1.set_ylim(-60, 60)
ax1.set_zlim(-60, 60)
ax1.set_aspect('equal','box')
plt.show()